# Testing Azure Functions

Testing Azure HTTP functions is easy because you can just use `requests` to ping the function. However EventHub functions are more difficult because you have to observe them happening in the context of the EventHub ecosystem. This usually means that you have to deploy and wait for your event to get triggered. 

Testing Azure Functions can be easy if you design your process right. The trick is to separate the Azure Function logic from the logic of your application. 

Like this:
```python
@app.function_name(name="myFunction")
@app.event_hub_message_trigger(arg_name="event",
                               event_hub_name=EVENT_HUB_NAME,
                               connection="EVENT_HUB_CONNECTION_STR")
def AZ_myFunction(event: func.EventHubEvent):
    eh_producer = EventHubProducerClient.from_connection_string(EVENT_HUB_CONNECTION_STR, eventhub_name=EVENT_HUB_NAME)
    credential = DefaultAzureCredential() 
    message = ast.literal_eval(event.get_body().decode('utf-8'))
    # proccessing messages is removed so that I can test it locally. 
    outgoing_messages = local_myFunction(message)
    if len(outgoing_messages)>0:
        logging.info(f"myFunction produced {len(outgoing_messages)} outgoing messages")
        send_to_eventhub(outgoing_messages, eh_producer)
        logging.info(f"Additional messages sent to EH. ")

def local_myFunction(message):
    # TODO: My business process that I can test locally
    outgoing_messages = []
    return outgoing_messages
```

This way I can deploy to the cloud, but I don't have to wait to see the results. I can deploy them locally and test my local business process. 

### A demonstration

In [1]:
import os
import  ssl, asyncio
import nest_asyncio
import pandas as pd

# moving back to the root, but idempotent in case we are already there
if 'function_app.py' not in os.listdir():
    os.chdir('..')
print([f for f in os.listdir() if f == 'function_app.py'])

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.info("logs showing as print")



ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

INFO:root:logs showing as print


['function_app.py']


In [2]:
import function_app as f

executing local windows deployment
something wrong with your query: <class 'Exception'>


## Generating the Messages to be Tested

Each Azure function has both a python function that is routed to the az function app, and a separate function that does the business logic of my game. This allows me to test locally.

| AZ Func Name | AZ Function | Relevant Local Function(args) | Description |
|----------|----------|----------|----------|
| actionResolverTimer | action_resolver | `process_action_messages()` | Queries open jobs and generates EventHub Messages to resolve them  |
| factionBuildingTimer | faction_building_resolver | `get_structure_messages()` | Structures that have ongoing effects |
| resolveActionEvents | resolve_action_event | `process_action_event_message(message)` | Takes individual event messages and resolves them |
| ututimer | utu_timer | `increment_timer()` | Increments the galatic timer |

In [17]:
action_messages = f.process_action_messages()

DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: health requirement 0.7
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: healthy_pops_query 0
INFO:root:EXOADMIN: No pops that meet the pop_health_requirement
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: Total jobs: 1
INFO:root:EXOADMIN: job instance of ACTION created: {'action': {'type': 'construction', 'comment': 'constructing a Shipyard', 'effort': 5, 'applies_to': 'pop', 'owned_by': 'pop', 'building': 'shipyard', 'created_at': 1007, 'to_build': '{type: shipyard, label: building, name: Shipyard, description: Stores ships for the faction , owned_by: pop, faction_augments: {wealth: -2}, requires_faction: {infrastructure: 10}, requires_attr: {industry: 0.1}, effort: 5}', 'objid': '1145817332658', 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'action'}, 'job': {'status': 'pending', 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'name': 'takingAction', 'weight': '1012

In [28]:
pd.DataFrame(action_messages)

,agent,action,job
0,"{'isIdle': 'false', 'name': 'Poldiasenah Ferfe...","{'type': 'construction', 'comment': 'construct...","{'status': 'pending', 'userguid': '8d5b667f-b2..."
1,"{'isIdle': 'false', 'name': 'Poldiasenah Des',...","{'type': 'construction', 'comment': 'construct...","{'status': 'pending', 'userguid': '8d5b667f-b2..."
2,"{'objid': '9888651460599', 'consumes': ['organ...",consume,NaN
3,"{'objid': '9629113611489', 'consumes': ['organ...",consume,NaN
4,"{'objid': '6668897649955', 'consumes': ['organ...",consume,NaN
5,"{'objid': '6880883763526', 'consumes': ['organ...",consume,NaN
6,"{'objid': '7372021564759', 'consumes': ['organ...",consume,NaN
7,"{'objid': '2416478665710', 'consumes': ['organ...",consume,NaN
8,"{'objid': '0255814826981', 'consumes': ['organ...",consume,NaN
9,"{'replenish_rate': 10, 'volume': 1044.0, 'obji...",renew,NaN


In [5]:
structure_messages = f.get_structure_messages()

DEBUG:asyncio:Using selector: SelectSelector
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: number of items: 4
INFO:root:EXOADMIN: Total Messages generated: 4 at: < time at UTU:1007 >


In [6]:
pd.DataFrame(structure_messages)

,faction,pop,structure,action
0,"{'wealth': 2.0, 'name': 'Poldiasenah', 'objid'...","{'health': '0.75', 'isIdle': 'true', 'name': '...","{'name': 'Commercial District', 'objid': '0799...",structure
1,"{'wealth': 2.0, 'name': 'Poldiasenah', 'objid'...","{'isIdle': 'true', 'name': 'Poldiasenah Bunar'...","{'name': 'Farmland', 'objid': '7148019480640',...",structure
2,"{'wealth': 2.0, 'name': 'Poldiasenah', 'objid'...","{'isIdle': 'true', 'name': 'Poldiasenah Ferfen...","{'name': 'Industrial manufacturing complex', '...",structure
3,"{'wealth': 2.0, 'name': 'Poldiasenah', 'objid'...","{'isIdle': 'true', 'name': 'Poldiasenah Des', ...","{'name': 'Commercial District', 'objid': '6336...",structure


## Resolving those messages

In [18]:
f.process_action_event_message(action_messages[0])

DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: processing message: {'type': 'construction', 'comment': 'constructing a Shipyard', 'effort': 5, 'applies_to': 'pop', 'owned_by': 'pop', 'building': 'shipyard', 'created_at': 1007, 'to_build': '{type: shipyard, label: building, name: Shipyard, description: Stores ships for the faction , owned_by: pop, faction_augments: {wealth: -2}, requires_faction: {infrastructure: 10}, requires_attr: {industry: 0.1}, effort: 5}', 'objid': '1145817332658', 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'action'} at UTU:< time at UTU:1014 >
INFO:root:EXOADMIN: job instance of ACTION created: {'agent': {'isIdle': 'false', 'name': 'Poldiasenah Des', 'objid': '6880883763526', 'conformity': 0.602, 'literacy': 0.625, 'aggression': 0.425, 'constitution': 0.482, 'health': 0.7, 'isIn': '4060880433595', 'industry': 0.4535, 'wealth': 0.5392, 'factionLoyalty': 0.673, 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'pop

[]

In [14]:
f.process_action_event_message(action_messages[1])

DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: processing message: {'type': 'construction', 'comment': 'constructing a Commercial District', 'effort': 1, 'applies_to': 'pop', 'owned_by': 'pop', 'building': 'commercial', 'created_at': 1002, 'to_build': '{type: commercial, label: building, name: Commercial District, description: Generates wealth for the faction., owned_by: pop, effort: 1, faction_augments: {wealth: 1}, requires_attr: {industry: 0.1}}', 'objid': '3190338367130', 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'action'} at UTU:< time at UTU:1004 >
INFO:root:EXOADMIN: job instance of ACTION created: {'agent': {'isIdle': 'false', 'name': 'Poldiasenah Sithane', 'objid': '9888651460599', 'conformity': 0.468, 'literacy': 0.631, 'aggression': 0.556, 'constitution': 0.372, 'health': 0.7, 'isIn': '4060880433595', 'industry': 0.464, 'wealth': 0.5475, 'factionLoyalty': 0.328, 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'pop'}, 'act

[]

In [17]:
pd.DataFrame(structure_messages)
# structure_messages[0]['faction']

,faction,pop,structure,action
0,"{'name': 'Poldiasenah', 'objid': '406088043359...","{'isIdle': 'true', 'name': 'Poldiasenah Sithan...","{'name': 'Commercial District', 'objid': '0799...",structure
1,"{'name': 'Poldiasenah', 'objid': '406088043359...","{'isIdle': 'true', 'name': 'Poldiasenah Bunar'...","{'name': 'Farmland', 'objid': '7148019480640',...",structure


In [7]:
f.process_action_event_message(structure_messages[1])

DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: processing message: structure at UTU:< time at UTU:1007 >
INFO:root:EXOADMIN:       -------And with that processed STRUCTURE: {'faction': {'wealth': 2.0, 'name': 'Poldiasenah', 'objid': '4060880433595', 'lat': 0.114, 'long': 0.0, 'infrastructure': 0, 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'faction', 'id': '4060880433595'}, 'pop': {'isIdle': 'true', 'name': 'Poldiasenah Bunar', 'objid': '9629113611489', 'conformity': 0.56, 'literacy': 0.5, 'aggression': 0.503, 'constitution': 0.448, 'health': 0.7, 'isIn': '4060880433595', 'industry': 0.4755, 'wealth': 0.4878, 'factionLoyalty': 0.531, 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'pop', 'id': '9629113611489'}, 'structure': {'name': 'Farmland', 'objid': '7148019480640', 'ownedBy': '9629113611489', 'type': 'farmland', 'description': 'Generates organic foodstuffs', 'populations will consume food before consuming natural resources': 'Non

[]

In [8]:
f.process_action_event_message(structure_messages[0])

DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: processing message: structure at UTU:< time at UTU:1007 >
INFO:root:EXOADMIN:       -------And with that processed STRUCTURE: {'faction': {'wealth': 2.0, 'name': 'Poldiasenah', 'objid': '4060880433595', 'lat': 0.114, 'long': 0.0, 'infrastructure': 0, 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'faction', 'id': '4060880433595'}, 'pop': {'health': '0.75', 'isIdle': 'true', 'name': 'Poldiasenah Sithane', 'objid': '9888651460599', 'conformity': 0.468, 'literacy': 0.631, 'aggression': 0.556, 'constitution': 0.372, 'isIn': '4060880433595', 'industry': 0.464, 'wealth': 0.5475, 'factionLoyalty': 0.328, 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'pop', 'id': '9888651460599'}, 'structure': {'name': 'Commercial District', 'objid': '0799411499496', 'ownedBy': '9888651460599', 'type': 'commercial', 'description': 'Generates wealth for the faction.', 'owned_by': 'pop', 'effort': 1, 'faction_augmen

[]

## Incrementing the timer
Just for testing purposes. Some jobs can only be done when ready. 


In [16]:
f.increment_timer()

DEBUG:asyncio:Using selector: SelectSelector
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: UTU was updated, result: currentTime was updated from:1013 to: 1014 at: < time at UTU:1013 >
